In [120]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time, datetime, os
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [142]:
NAME = "264x32x16x1_MSE_sigmoid_glo_6ep{}".format(int(time.time()))

log_dir = "logs/fit/" + NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stats = pd.read_csv(r'Data\final_dataset.csv')
stats = stats.drop('Score',1)

**Custom Loss Function**
Keras does not allow to give any parameter other than y_true and y_pred, to get around we encode the odds in the y_true tensor.

In [143]:
X = stats.iloc[:,3:-3]
y = stats.iloc[:,-1:]
#y = stats.iloc[:,-3:]
#y = tf.convert_to_tensor(y)
def custom_loss(y_true, y_pred):
    """
    y_true = (batch_size, output nodes)
    y_pred = (batch_size, output nodes)
    """
    odds1 = y_true[:, 0:1]
    odds2 = y_true[:, 1:2]
    y_true = y_true[:, :-1]
    loss = -1 * K.sum((odds1 * y_true -1) * K.log(odds1 * y_pred -1) + (odds2 * (1- y_true) -1) * K.log(odds2 * (1-y_pred)-1))
    
    return loss
    

random_state = 12
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state=random_state)
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)


In [144]:
model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(16, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(1, kernel_initializer='GlorotNormal', activation='sigmoid'))

model.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 5, epochs=6, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

Epoch 1/6
2288/2288 [==============================] - 2s 944us/step - loss: 0.2220 - accuracy: 0.6450 - val_loss: 0.2135 - val_accuracy: 0.6563
Epoch 2/6
2288/2288 [==============================] - 2s 740us/step - loss: 0.2171 - accuracy: 0.6555 - val_loss: 0.2132 - val_accuracy: 0.6657
Epoch 3/6
2288/2288 [==============================] - 2s 746us/step - loss: 0.2149 - accuracy: 0.6619 - val_loss: 0.2117 - val_accuracy: 0.6748
Epoch 4/6
2288/2288 [==============================] - 2s 737us/step - loss: 0.2131 - accuracy: 0.6659 - val_loss: 0.2110 - val_accuracy: 0.6689
Epoch 5/6
2288/2288 [==============================] - 2s 777us/step - loss: 0.2114 - accuracy: 0.6706 - val_loss: 0.2116 - val_accuracy: 0.6741
Epoch 6/6
2288/2288 [==============================] - 2s 748us/step - loss: 0.2091 - accuracy: 0.6701 - val_loss: 0.2114 - val_accuracy: 0.6759


In [145]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

90/90 [==============================] - 0s 740us/step - loss: 0.2114 - accuracy: 0.6759
0.2114359438419342 0.6758741140365601


In [146]:
print("Generate a prediction")
prediction = model.predict(X_test[:20])
print("prediction:", prediction)
print(y_test[:20]) 


Generate a prediction
prediction: [[0.19128305]
 [0.43825045]
 [0.2619453 ]
 [0.22829315]
 [0.452913  ]
 [0.55657303]
 [0.74520147]
 [0.48667052]
 [0.45464498]
 [0.6408756 ]
 [0.6918521 ]
 [0.79550266]
 [0.61743057]
 [0.58354485]
 [0.2593447 ]
 [0.29844815]
 [0.25415578]
 [0.3739645 ]
 [0.64564496]
 [0.5540498 ]]
       Results
12047      1.0
5092       0.0
4915       1.0
7563       0.0
6054       0.0
5913       1.0
10748      1.0
4061       1.0
8971       1.0
6588       1.0
6381       0.0
13380      1.0
2109       1.0
7189       1.0
5750       0.0
636        1.0
7703       0.0
3242       0.0
3803       0.0
8766       0.0


In [147]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 18168), started 2:03:48 ago. (Use '!kill 18168' to kill it.)

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
_sc = StandardScaler()
_pca = PCA(n_components = None)
_model = LogisticRegression()
log_regress_model = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
('regressor', _model)])

In [33]:
# perform a split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.3,
                     shuffle=True, 
                     random_state=random_state)
# train the model using the PCA components
log_regress_model.fit(X_train,y_train)



C:\Users\Bei\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Bei\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('std_scaler', StandardScaler()), ('pca', PCA()),
                ('regressor', LogisticRegression())])

In [34]:
log_regress_model.score(X_test,y_test)

0.6631701631701632